In [1]:
import pandas as pd
import numpy as np
import pulp
import os
os.getcwd()
os.chdir("..")

# Process data

In [2]:
raw_workmode = pd.read_excel('InputData.xlsx', sheet_name="Mode")
raw_demand = pd.read_excel('InputData.xlsx', sheet_name="DemandV2") # ver2
workmode = {(row['Mode'].item(), row['Shift'].item()): row['Capa'].item() for _, row in raw_workmode.iterrows()}
demand = {row['Week'].item(): row['Demand'].item() for _, row in raw_demand.iterrows()} # ver2

In [3]:
WeekIndex = raw_demand.Week.tolist()
ShiftIndex = [1,2,3]
bigM = 99999

In [4]:
WeekIndex

[1, 2, 3, 4, 5, 6, 7, 8]

In [5]:
numS = 22
numC = 12
FC = {
    (i, j): raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].item() * (8.3 * 1.5 if j in [2, 3] else 8.3)
    for i in [1, 2]
    for j in ShiftIndex
    if not raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].empty}

# Algorithm

In [6]:
# Create the model
model = pulp.LpProblem("3PL_Workforce_Allocation", pulp.LpMinimize)

# Define binary decision variables x (3D) and y (2D)
m_conso = pulp.LpVariable.dicts("m_conso", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')
m_sepa = pulp.LpVariable.dicts("m_sepa", (ShiftIndex, WeekIndex), lowBound=0, cat='Integer')

# Objective function: 
model += pulp.lpSum(numC*FC[1, sh] * m_conso[sh][w] for sh in ShiftIndex for w in WeekIndex) + \
         pulp.lpSum(numS*FC[2, sh] * m_sepa[sh][w] for sh in ShiftIndex for w in WeekIndex)

# Constraint

# Can't work more than 3 shifts 3 per week
for w in WeekIndex:
    model += m_conso[3][w] + m_sepa[3][w] <= 3

# Satisfy demand ver 2
for w in WeekIndex:
    model += pulp.lpSum(workmode[1, sh]*m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(workmode[2, sh]*m_sepa[sh][w] for sh in ShiftIndex) >= demand[w]

# One shift only within a day
# for w in WeekIndex:
#     for d in WeekDayIndex:
#         model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex) == 1

# Min 5 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) >= 5

# Max 7 days a week
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w] for sh in ShiftIndex) <= 7

# Always larger than 0
        

# Solve the problem using COIN_CMD without the tol argument
solver = pulp.PULP_CBC_CMD(gapRel=0.00001)  # Built-in solver
model.solve(solver)

# Collect the results into a list of dictionaries for x_ijk (3D)
results_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_conso.append({
            'shift': sh,
            'week': w,
            'choose': m_conso[sh][w].varValue
        })

# Collect the results into a list of dictionaries for y_ij (2D)
results_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        results_sepa.append({
            'shift': sh,
            'week': w,
            'choose': m_sepa[sh][w].varValue
        })

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lamjackie/miniconda3/envs/ych-wf-allocation/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/529a5d25a4244168b0d5945baf8f6962-pulp.mps -ratio 1e-05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/529a5d25a4244168b0d5945baf8f6962-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 37 COLUMNS
At line 342 RHS
At line 375 BOUNDS
At line 424 ENDATA
Problem MODEL has 32 rows, 48 columns and 160 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1e-05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 69324.3 - 0.00 seconds
Cgl0004I processed model has 24 rows, 48 columns (48 integer (0 of which binary)) and 112 elements
Cutoff increment increased from 1e-05 to 16.5999
Cbc0012I Integer solution of 7136

In [7]:
df_results_conso=pd.DataFrame(results_conso)
df_results_conso['mode'] = "c"
df_results_sepa = pd.DataFrame(results_sepa)
df_results_sepa['mode'] = "s"
result = pd.concat([df_results_sepa, df_results_conso])
result_pivot=pd.DataFrame(result.pivot_table(index=["mode", "shift"], columns=["week"], values=["choose"], aggfunc="sum").reset_index(drop=False))
result_pivot

mode shift choose                                   
week                 1    2    3    4    5    6    7    8
0       c     1    1.0  0.0  0.0  3.0  4.0  5.0  7.0  5.0
1       c     2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       c     3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3       s     1    6.0  3.0  5.0  3.0  3.0  2.0  0.0  0.0
4       s     2    0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
5       s     3    0.0  3.0  2.0  0.0  0.0  0.0  0.0  0.0

In [ ]:
workmode

In [ ]:
raw_demand

In [9]:
with pd.ExcelWriter("result_v2.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    result_pivot.to_excel(writer, sheet_name="Pivot")
    result.to_excel(writer, sheet_name="Results", index=False)
    raw_workmode.to_excel(writer, sheet_name="Workforce", index=False)
    raw_demand.to_excel(writer, sheet_name="Demand", index=False)

In [8]:
df_results_sepa

,shift,week,choose,mode
0,1,1,6.0,s
1,1,2,3.0,s
2,1,3,5.0,s
3,1,4,3.0,s
4,1,5,3.0,s
5,1,6,2.0,s
6,1,7,0.0,s
7,1,8,0.0,s
8,2,1,0.0,s
9,2,2,1.0,s
